# 一、数据生成器
## 1、自建数据生成器
自建数据生成器仅支持二分类，建议使用API数据生成器。对于二分类问题，将下面代码中的两个路径正则匹配好即可<br>
```
real_list  = glob.glob(r".\Hotdog\train\hotdog/*.png")  
glitch_list = glob.glob(r".\Hotdog\train\not-hotdog/*png")
```

In [ ]:
import glob

# real_list  = glob.glob(r".\train\1/*.png")
# glitch_list = glob.glob(r".\train\0/*png")

real_list  = glob.glob(r".\Hotdog\train\hotdog/*.png")  
glitch_list = glob.glob(r".\Hotdog\train\not-hotdog/*png")

list_img = real_list+glitch_list

print(len(real_list), len(glitch_list))

In [ ]:
import cv2
import numpy as np
import pprint as pp

def get_batch(img_list, label_list, batch_size=8, show=False):
    # 在方法中需要用while写成死循环，因为每个step不会重新调用方法
    while True:
        index = np.random.choice(range(len(img_list)), batch_size)
        batch_list = [img_list[i] for i in index]
        batch_row = [cv2.imread(img) for img in batch_list]
        batch_gray = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in batch_row]
        batch_matrix = [cv2.resize(i,(224, 224),interpolation=cv2.INTER_CUBIC) for i in batch_gray]
        batch_label = [label_list[i] for i in index]
        if show:
            pp.pprint(list(zip(batch_list, batch_label)))
        batch_matrix = np.concatenate(np.expand_dims(batch_matrix, axis=0), axis=0).astype(float)
        batch_label = np.array(batch_label)
        # batch_matrix = np.expand_dims(batch_matrix, -1)
        batch_matrix = (batch_matrix).astype(int)/255.
        yield batch_matrix, batch_label  # , batch_list


list_lab = np.zeros([len(list_img), 2], dtype=int)
list_lab[:len(real_list), 0] = 1  # real: [1, 0]
list_lab[len(real_list):, 1] = 1  # fake: [0, 1]
b = next(get_batch(list_img, list_lab, show=True))

In [ ]:
import matplotlib.pyplot as plt
num = np.random.randint(0, len(b))
img = (b[0][num]*255).astype(np.uint8) 
plt.imshow(img)
plt.show()
# print(b[2][num])
# pp.pprint(list(zip(b[1][:], b[2][:])))

## 2、API数据生成器
官方的数据读取器，路径指定到图片父级目录，其下有各个class的子目录即可

In [1]:
from keras.preprocessing.image import ImageDataGenerator

# train_datagen=ImageDataGenerator(rescale=1./255)
# train_flow=train_pic_gen.flow_from_directory(r'./hotdog\train',target_size=(128, 128),batch_size=32,class_mode='categorical')

train_datagen = ImageDataGenerator(
        rescale=1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False)

train_flow = train_datagen.flow_from_directory(
        r'.\Hotdog\train',            # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical')

Using TensorFlow backend.


Found 2000 images belonging to 2 classes.


# 二、网络定义


In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense,Flatten,Dropout,Input
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import InceptionV3

TransforLearning = True
model = Sequential()
if not TransforLearning:
    model.add(Conv2D(32, 3, 3, input_shape=(224, 224, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())# this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('sigmoid'))
    
else:
    inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))
    model.add(inception)
    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    top_model.add(Flatten(input_shape=model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(2, activation='sigmoid'))

    # 数据集不够大，这里将inception部分参数冻结，不参与训练
    model.add(top_model)
#     for layer in inception.layers[:]:
#         layer.trainable = False

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
sequential_2 (Sequential)    (None, 2)                 13107970  
Total params: 34,910,754
Trainable params: 34,876,322
Non-trainable params: 34,432
_________________________________________________________________


# 三、训练

In [3]:
from keras import losses
import keras.backend as K
from keras import optimizers
from keras.callbacks import TensorBoard

samples_per_epoch = 10000  # len(list_img)
batch_size = 32

# K.set_value(sgd.lr, 0.5 * K.get_value(sgd.lr))
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              loss='binary_crossentropy',  # 'categorical_crossentropy',
              metrics=['accuracy'])


model.fit_generator(train_flow,  
                    # get_batch(list_img, list_lab),
                    steps_per_epoch=2*samples_per_epoch//batch_size, # samples_per_epoch//batch_size, 
                    epochs=3,
                    shuffle=True,
                    callbacks=[TensorBoard(log_dir='./tmp/log', write_graph=True)])
# model.fit(img, label, batch_size=8, epochs=10)
model.save('model.h5')

Epoch 1/3
625/625 [==============================] - 353s 565ms/step - loss: 0.3203 - acc: 0.8532
Epoch 2/3
625/625 [==============================] - 351s 562ms/step - loss: 0.0984 - acc: 0.9636
Epoch 3/3
625/625 [==============================] - 351s 562ms/step - loss: 0.0479 - acc: 0.9845


# 四、测试
## 1、自建数据生成器
如果上面用的get_batch函数获取的训练数据，则这里也要使用自建的数据获取流程<br>
须将下面两个路径替换为测试图片的路径：
```
test_real_list  = glob.glob("./0/*.jpg")
test_glitch_list = glob.glob("./1/*jpg")
```

In [13]:
import matplotlib.pyplot as plt

test_real_list  = glob.glob("./test/0/*.png")
test_glitch_list = glob.glob("./test/1/*.png")
test_list_img = test_real_list + test_glitch_list

img_test = [cv2.imread(i) for i in test_list_img]

In [26]:
img_test = [cv2.resize(i,(224, 224),interpolation=cv2.INTER_CUBIC)
            for i in img_test if not i==None]
img_test = np.concatenate(np.expand_dims(img_test, axis=0), axis=0)/255

label_test = np.zeros([len(img_test), 2], dtype=int)
label_test[:len(test_real_list), 0] = 1
label_test[len(test_real_list):, 1] = 1
pp.pprint(list(zip(test_list_img, label_test)))

C:\software\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  


[('./test/0\\fake_0_50_0_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_10_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_11_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_12_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_13_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_14_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_15_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_16_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_17_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_18_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_19_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_1_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_2_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_3_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_4_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_5_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_6_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_7_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_8_0.png', array([1, 0])),
 ('./test/0\\fake_0_50_9_0.png', array([1, 0])),
 ('./test/

 ('./test/0\\fake_4_45_17_0.png', array([1, 0])),
 ('./test/0\\fake_4_45_18_0.png', array([1, 0])),
 ('./test/0\\fake_4_45_19_0.png', array([1, 0])),
 ('./test/0\\fake_4_45_1_0.png', array([1, 0])),
 ('./test/0\\fake_4_45_2_0.png', array([1, 0])),
 ('./test/0\\fake_4_45_3_0.png', array([1, 0])),
 ('./test/0\\fake_4_45_4_0.png', array([1, 0])),
 ('./test/0\\fake_4_45_5_0.png', array([1, 0])),
 ('./test/0\\fake_4_45_6_0.png', array([1, 0])),
 ('./test/0\\fake_4_45_7_0.png', array([1, 0])),
 ('./test/0\\fake_4_45_8_0.png', array([1, 0])),
 ('./test/0\\fake_4_45_9_0.png', array([1, 0])),
 ('./test/0\\fake_4_54_0_0.png', array([1, 0])),
 ('./test/0\\fake_4_54_10_0.png', array([1, 0])),
 ('./test/0\\fake_4_54_11_0.png', array([1, 0])),
 ('./test/0\\fake_4_54_12_0.png', array([1, 0])),
 ('./test/0\\fake_4_54_13_0.png', array([1, 0])),
 ('./test/0\\fake_4_54_14_0.png', array([1, 0])),
 ('./test/0\\fake_4_54_15_0.png', array([1, 0])),
 ('./test/0\\fake_4_54_16_0.png', array([1, 0])),
 ('./test/

## 2、API数据生成器
否则使用迭代器即可

In [4]:
test_datagen = ImageDataGenerator(
        rescale=1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False)

test_flow = train_datagen.flow_from_directory(
        r'.\Hotdog\test',            # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical')

Found 800 images belonging to 2 classes.


In [5]:
import keras.models as KM

model = KM.load_model('model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
sequential_2 (Sequential)    (None, 2)                 13107970  
Total params: 34,910,754
Trainable params: 34,876,322
Non-trainable params: 34,432
_________________________________________________________________


In [7]:
import pprint as pp
# result = model.evaluate(img_test, label_test)  # 
result = model.evaluate_generator(test_flow, 1)
pp.pprint(list(zip(model.metrics_names, result)))

[('loss', 0.021519791334867477), ('acc', 1.0)]
